In [1]:
import emcee
from scipy.optimize import minimize
import corner
import ptemcee
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from emukit.core import ParameterSpace, ContinuousParameter
from GPy.models import MultioutputGP
from GPy import likelihoods
from emukit.model_wrappers import GPyMultiOutputWrapper
from GPy.kern import RBF

1-dimensional input, 2-dimensional outputs

In [2]:
parameter_space = ParameterSpace([ContinuousParameter('x', 0, 1)])

In [3]:
from emukit.core.initial_designs import RandomDesign

design = RandomDesign(parameter_space)
num_data_points = 2
X = design.get_samples(num_data_points)

In [4]:
def f(X): # add noise
    y1 = np.sin(X)
    y2 = np.cos(X)
    return np.hstack([np.sin(X),np.cos(X)])

In [5]:
X

array([[0.4778312 ],
       [0.25618252]])

Predict 1 point:

Use 0,1 to indicate which output to predict:

In [6]:
xtest = np.array([[0.5],[0.5]])
new_column = [[0], [1]]
xtest = np.append(xtest, new_column, axis=1)
xtest

array([[0.5, 0. ],
       [0.5, 1. ]])

Add indicator to training points:

In [7]:
from emukit.multi_fidelity.convert_lists_to_array import convert_xy_lists_to_arrays

In [8]:
X_train, Y_train = convert_xy_lists_to_arrays([X,X],[np.sin(X),np.cos(X)])
[X_train, Y_train]

[array([[0.4778312 , 0.        ],
        [0.25618252, 0.        ],
        [0.4778312 , 1.        ],
        [0.25618252, 1.        ]]),
 array([[0.45985438],
        [0.25338951],
        [0.88799434],
        [0.96736433]])]

In [16]:
kernel1 = RBF(input_dim=1)
kernel2 = RBF(input_dim=1)
model_gpy = MultioutputGP([X,X],[np.sin(X),np.cos(X)],
                          likelihood_list=[likelihoods.Gaussian(),likelihoods.Gaussian()],
                          kernel_list=[kernel1,kernel2])
model_emukit = GPyMultiOutputWrapper(model_gpy, n_outputs=2, n_optimization_restarts=10)

In [17]:
model_emukit.optimize()

Optimization restart 1/10, f = 0.4583460194830993
Optimization restart 2/10, f = 0.45834601930101293
Optimization restart 3/10, f = 0.45834601947268716
Optimization restart 4/10, f = 0.45834601938853403
Optimization restart 5/10, f = 0.458346019255937
Optimization restart 6/10, f = 0.45834601930338437
Optimization restart 7/10, f = 0.45834601925918506
Optimization restart 8/10, f = 0.4583460192566724
Optimization restart 9/10, f = 0.4583460192567541
Optimization restart 10/10, f = 0.4583460192576956


In [18]:
xtest

array([[0.5, 0. ],
       [0.5, 1. ]])

In [19]:
model_emukit.predict(xtest)

(array([[0.43167024],
        [0.88898683]]),
 array([[0.01191372],
        [0.00131162]]))

TO UPDATE

In [ ]:
from emukit.experimental_design.acquisitions.contour_MultiObs import contourWeightedEI

In [ ]:
a_list = [0.2, 0.5]
alpha_list = [2, 2]
w_list = [0.5, 0.5]

In [ ]:
contour_MD = contourWeightedEI(models = model_emukit, a_list = a_list, alpha_list = alpha_list, w_list = w_list)